In [1]:
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler

/Users/anusar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
import urllib.request
urllib.request.urlretrieve ("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", "a.tar.gz")

('a.tar.gz', <http.client.HTTPMessage at 0x7ff88bc7bfd0>)

In [0]:
import tarfile
tar = tarfile.open("a.tar.gz")
tar.extractall()
tar.close()

## Loading Dataset from link

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pickle


def unpickle(file):
 '''Load byte data from file'''
 with open(file, 'rb') as f:
  data = pickle.load(f, encoding='latin-1')
  return data


def load_cifar10_data(data_dir):
 '''Return train_data, train_labels, test_data, test_labels
 The shape of data is 32 x 32 x3'''
 train_data = None
 train_labels = []

 for i in range(1, 6):
  data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
  if i == 1:
   train_data = data_dic['data']
  else:
   train_data = np.vstack((train_data, data_dic['data']))
  train_labels += data_dic['labels']

 test_data_dic = unpickle(data_dir + "/test_batch")
 test_data = test_data_dic['data']
 test_labels = test_data_dic['labels']

 train_data = train_data.reshape((len(train_data), 3, 32, 32))
 train_data = np.rollaxis(train_data, 1, 4)
 train_labels = np.array(train_labels)

 test_data = test_data.reshape((len(test_data), 3, 32, 32))
 test_data = np.rollaxis(test_data, 1, 4)
 test_labels = np.array(test_labels)

 return train_data, train_labels, test_data, test_labels

data_dir = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(data_dir)

print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)


(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [0]:
x_train = train_data
x_test = test_data
y_train = train_labels
y_test = test_labels

## Scaling the Data

In [25]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train.reshape(50000,-1))
x_test = scaler.transform(x_test.reshape(10000,-1))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:
x_train = x_train.reshape(50000, 32, 32, 3)
x_test = x_test.reshape(10000, 32, 32, 3)

In [27]:
x_train.shape

(50000, 32, 32, 3)

## One Hot Encoding Labels

In [0]:
from keras.utils import np_utils
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [0]:
n_classes = 10
img_size = 32
channels = 3

## Creating Model

In [0]:
from keras.layers import Conv2D, MaxPooling2D, Activation , Dense , Dropout , Flatten
from keras.callbacks import ModelCheckpoint

In [0]:
from keras.models import Sequential

In [0]:
def cnn_model():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_size,img_size,channels)))    
    model.add(Conv2D(32, (3, 3), activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(num_classes, activation='softmax'))
    
    model.summary()
    
    return model

In [33]:
model = cnn_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
__________

## Training the Model

In [0]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr= 0.001),
              metrics = ['accuracy'])

In [35]:
model.fit(x_train,y_train,batch_size=128,epochs=10,validation_data=(x_test,y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 14s 284us/step - loss: 1.4622 - acc: 0.4694 - val_loss: 1.1532 - val_acc: 0.5845
Epoch 2/10
50000/50000 [==============================] - 13s 267us/step - loss: 1.0519 - acc: 0.6261 - val_loss: 0.8831 - val_acc: 0.6938
Epoch 3/10
50000/50000 [==============================] - 13s 267us/step - loss: 0.8512 - acc: 0.6985 - val_loss: 0.7892 - val_acc: 0.7203
Epoch 4/10
50000/50000 [==============================] - 13s 266us/step - loss: 0.7291 - acc: 0.7447 - val_loss: 0.7211 - val_acc: 0.7492
Epoch 5/10
50000/50000 [==============================] - 13s 267us/step - loss: 0.6301 - acc: 0.7799 - val_loss: 0.6867 - val_acc: 0.7659
Epoch 6/10
50000/50000 [==============================] - 13s 268us/step - loss: 0.5513 - acc: 0.8074 - val_loss: 0.6540 - val_acc: 0.7762
Epoch 7/10
50000/50000 [==============================] - 13s 267us/step - loss: 0.4779 - acc: 0.8326 - val_loss: 0.

## Accuracy - 77.61%

We can get 77.61% accuracy by using simple CNN on cifar-10 dataset